In [3]:
import json
import ast

ll = []
with open('latlong.json') as data_file:    
    for line in data_file:
        entry = ast.literal_eval(line)
        del entry['addresses_associated']
        del entry['_id']
        ll.append(entry)

print(ll[1])

{'collaped_address': '300 NEW TORONTO ST', 'num_associated': 5, 'latitude': 43.61593765, 'longitude': -79.321369}


In [4]:
import json
with open('latlong_fixed.json', 'w') as outfile:
    json.dump(ll, outfile)

In [1]:
#get REAL num associated
import csv
import re
import sys
from collections import Counter

csv.field_size_limit(sys.maxsize)

#make list of all addresses
years=['2008','2009','2010','2011','2012','2013','2014','2015']
files_to_open=['data/'+year+'.csv' for year in years]

addresses = []
def get_all_addresses(years, files_to_open):
    for file in files_to_open:
        try:
            with open(file,'rt',encoding='utf8') as f:
                reader = csv.DictReader(f)
                for line in reader:
                    addresses.append(line['location2'])
                print(file+" success")
        except FileNotFoundError:
            print(file+" failed")
            
get_all_addresses(years,files_to_open)

#create counter object of addresses
address_counter = Counter(addresses)

data/2008.csv success
data/2009.csv success
data/2010.csv success
data/2011.csv success
data/2012.csv success
data/2013.csv success
data/2014.csv success
data/2015.csv success


In [2]:
import pickle
#addresses per collapsed address
collapsed_address_count={}
cam = pickle.load(open("collapsed_address_map.pkl", "rb" ))
for key in cam.keys():
    count = 0
    for address in cam[key]:
        count+=address_counter[address]
    collapsed_address_count[key]=count

In [3]:
with open('collapsed_address_count.pkl', 'wb') as file:
    pickle.dump(collapsed_address_count, file)

In [5]:
print(len(collapsed_address_count.keys()))

74224


In [ ]:
from pymongo import MongoClient

dbconn = MongoClient()
db = dbconn.parking.latlong_copy

for key in collapsed_address_count:
    try:
        db.find_one_and_update({'collaped_address':key},{'$set':{'num_associated':collapsed_address_count[key]}})
    except:
        print(key)